# Day 09. Exercise 01
# Gridsearch

## 0. Imports

In [12]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score

## 1. Preprocessing

1. Read the file [`day-of-week-not-scaled.csv`](https://drive.google.com/file/d/1AlGvsJDSzPT_70caausx8bFuupIEZkfh/view?usp=sharing). It is similar to the one from the previous exercise, but this time we did not scale continuous features (we are not going to use logreg anymore).
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test`. Use the additional parameter `stratify`.

In [16]:
df = pd.read_csv('data/dayofweek-not-scaled.csv')
df

,numTrials,hour,dayofweek,uid_user_0,uid_user_1,uid_user_10,uid_user_11,uid_user_12,uid_user_13,uid_user_14,...,labname_lab02,labname_lab03,labname_lab03s,labname_lab05s,labname_laba04,labname_laba04s,labname_laba05,labname_laba06,labname_laba06s,labname_project1
0,1,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,2,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,4,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,5,5,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1681,9,20,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1682,6,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1683,7,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1684,8,20,3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('dayofweek', axis=1), df['dayofweek'],
    test_size=0.2, random_state=21, stratify=df['dayofweek']
    )

## 2. SVM gridsearch

1. Using `GridSearchCV` try different parameters of kernel (`linear`, `rbf`, `sigmoid`), C (`0.01`, `0.1`, `1`, `1.5`, `5`, `10`), gamma (`scale`, `auto`), class_weight (`balanced`, `None`) use `random_state=21` and `probability=True` and get the best combination of them in terms of accuracy.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`. Check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [22]:
svc = SVC(probability=True, random_state=21)

In [24]:
param_grid = {'C': [0.01, 0.1, 1, 1.5, 5, 10],
              'kernel': ['linear', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'class_weight': ['balanced', None]
              }

gs = GridSearchCV(svc, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=SVC(probability=True, random_state=21), n_jobs=-1,
             param_grid={'C': [0.01, 0.1, 1, 1.5, 5, 10],
                         'class_weight': ['balanced', None],
                         'gamma': ['scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'sigmoid']},
             scoring='accuracy')

In [30]:
gs.best_params_

{'C': 10, 'class_weight': None, 'gamma': 'auto', 'kernel': 'rbf'}

In [32]:
gs.best_score_

0.8761090458488228

In [34]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [36]:
results.head(7)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
70,1.628329,0.226393,0.091379,0.007015,10.0,None,auto,rbf,"{'C': 10, 'class_weight': None, 'gamma': 'auto...",0.900000,0.848148,0.885185,0.884758,0.862454,0.876109,0.018419,1
64,1.902311,0.170944,0.121880,0.036448,10.0,balanced,auto,rbf,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.877778,0.851852,0.862963,0.873606,0.851301,0.863500,0.010870,2
58,1.530668,0.127085,0.106927,0.013351,5.0,None,auto,rbf,"{'C': 5, 'class_weight': None, 'gamma': 'auto'...",0.825926,0.811111,0.818519,0.821561,0.802974,0.816018,0.008116,3
52,1.777987,0.323169,0.110269,0.010788,5.0,balanced,auto,rbf,"{'C': 5, 'class_weight': 'balanced', 'gamma': ...",0.844444,0.785185,0.792593,0.817844,0.802974,0.808608,0.021007,4
63,173.097027,19.034546,0.025084,0.003659,10.0,balanced,auto,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5
60,175.573231,14.858137,0.028155,0.007963,10.0,balanced,scale,linear,"{'C': 10, 'class_weight': 'balanced', 'gamma':...",0.729630,0.700000,0.755556,0.754647,0.665428,0.721052,0.034438,5
66,132.403233,17.520227,0.022194,0.000386,10.0,None,scale,linear,"{'C': 10, 'class_weight': None, 'gamma': 'scal...",0.737037,0.711111,0.707407,0.743494,0.698885,0.719587,0.017463,7


## 3. Decision tree

1. Using `GridSearchCV` try different parameters of `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use `random_state=21`.
2. Create a dataframe from the results of the gridsearch and sort it ascendingly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [40]:
tree = DecisionTreeClassifier(random_state=21)

In [42]:
param_grid = {'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None]
            }

gs = GridSearchCV(tree, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49])},
             scoring='accuracy')

In [44]:
gs.best_params_

{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 22}

In [46]:
gs.best_score_

0.8731212997384002

In [48]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [50]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
21,0.011510,0.000924,0.003520,0.000698,balanced,gini,22,"{'class_weight': 'balanced', 'criterion': 'gin...",0.885185,0.862963,0.903704,0.881041,0.832714,0.873121,0.023998,1
20,0.011744,0.000716,0.003600,0.000590,balanced,gini,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.859259,0.903704,0.884758,0.828996,0.873121,0.026300,2
29,0.011230,0.000505,0.003451,0.000398,balanced,gini,30,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
32,0.011764,0.000448,0.003420,0.000369,balanced,gini,33,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
28,0.011016,0.000556,0.003245,0.000846,balanced,gini,29,"{'class_weight': 'balanced', 'criterion': 'gin...",0.888889,0.866667,0.903704,0.873606,0.832714,0.873116,0.023911,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,0.008812,0.001452,0.003924,0.000644,balanced,gini,3,"{'class_weight': 'balanced', 'criterion': 'gin...",0.388889,0.303704,0.403704,0.427509,0.345725,0.373906,0.044064,192
98,0.007327,0.003379,0.004198,0.001027,None,gini,1,"{'class_weight': None, 'criterion': 'gini', 'm...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
147,0.005896,0.000586,0.003409,0.000379,None,entropy,1,"{'class_weight': None, 'criterion': 'entropy',...",0.370370,0.351852,0.359259,0.353160,0.342007,0.355330,0.009338,193
49,0.006526,0.000329,0.002894,0.000491,balanced,entropy,1,"{'class_weight': 'balanced', 'criterion': 'ent...",0.262963,0.318519,0.266667,0.323420,0.260223,0.286358,0.028376,195


## 4. Random forest

1. Using `GridSearchCV` try different parameters of `n_estimators` (`5`, `10`, `50`, `100`), `max_depth` (from `1` to `49`), `class_weight` (`balanced`, `None`) and `criterion` (`entropy` and `gini`) and get the best combination of them in terms of accuracy. Use random_state=21.
2. Create a dataframe from the results of the gridsearch and sort it ascendengly by the `rank_test_score`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [54]:
frt = RandomForestClassifier(random_state=21)

In [60]:
param_grid = {'n_estimators': [5, 10, 50, 100],
              'criterion': ['gini','entropy'],
              'max_depth': np.arange(1, 50),
              'class_weight': ['balanced', None],
              'random_state': [21]}

gs = GridSearchCV(frt, param_grid, scoring='accuracy', n_jobs=-1)
gs.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=21), n_jobs=-1,
             param_grid={'class_weight': ['balanced', None],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),
                         'n_estimators': [5, 10, 50, 100],
                         'random_state': [21]},
             scoring='accuracy')

In [62]:
gs.best_params_

{'class_weight': None,
 'criterion': 'gini',
 'max_depth': 28,
 'n_estimators': 50,
 'random_state': 21}

In [64]:
gs.best_score_

0.9042902381935839

In [66]:
results = pd.DataFrame(gs.cv_results_)
results = results.sort_values('rank_test_score', ascending=True)

In [68]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_class_weight,param_criterion,param_max_depth,param_n_estimators,param_random_state,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
502,0.272397,0.041701,0.013219,0.005181,None,gini,28,50,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.922222,0.900000,0.907407,0.903346,0.888476,0.904290,0.010961,1
515,0.671498,0.093170,0.034992,0.000758,None,gini,31,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.918519,0.911111,0.900000,0.910781,0.877323,0.903547,0.014380,2
118,0.417177,0.014386,0.021084,0.000835,balanced,gini,30,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.881481,0.907063,0.895911,0.902817,0.013554,3
134,0.446274,0.082692,0.025645,0.006460,balanced,gini,34,50,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.922222,0.907407,0.892593,0.907063,0.884758,0.902809,0.013010,4
587,0.807231,0.052975,0.038171,0.001818,None,gini,49,100,21,"{'class_weight': None, 'criterion': 'gini', 'm...",0.914815,0.911111,0.900000,0.903346,0.884758,0.902806,0.010460,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,0.039576,0.001672,0.008523,0.000929,None,entropy,1,5,21,"{'class_weight': None, 'criterion': 'entropy',...",0.355556,0.366667,0.374074,0.345725,0.327138,0.353832,0.016467,780
200,0.040107,0.000772,0.008226,0.000427,balanced,entropy,2,5,21,"{'class_weight': 'balanced', 'criterion': 'ent...",0.318519,0.366667,0.381481,0.353160,0.345725,0.353110,0.021165,781
4,0.041825,0.003147,0.008941,0.000422,balanced,gini,2,5,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.311111,0.377778,0.377778,0.353160,0.312268,0.346419,0.029749,782
0,0.035773,0.005299,0.007903,0.001482,balanced,gini,1,5,21,"{'class_weight': 'balanced', 'criterion': 'gin...",0.262963,0.292593,0.285185,0.282528,0.293680,0.283390,0.011062,783


## 5. Progress bar

Gridsearch can be a quite long process and you may find yourself wondering when it will end.
1. Create a manual gridsearch for the same parameters values of random forest iterating through the list of the possible values and calculating `cross_val_score` for each combination. Try to increase `n_jobs`. The value `cv` for `cross_val_score` is 5.
2. Track the progress using the library `tqdm.notebook`.
3. Create a dataframe from the results of the gridsearch with the columns corresponding to the names of the parameters and `mean_accuracy` and `std_accuracy`.
4. Sort it descendingly by the `mean_accuracy`, check if there is a huge difference between different combinations (sometimes a simpler model may give a comparable result).

In [72]:
data = []

for params in tqdm(product(*param_grid.values()), total=len(list(product(*param_grid.values()))), desc='Grid Search'):
    current_params = {}
    for param_name, value in zip(param_grid.keys(), params):
        current_params[param_name] = value

    # Define the model with current parameters
    rf = RandomForestClassifier(**current_params)

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=21)
    # Calculate cross-validated accuracy
    mean_accuracy = cross_val_score(rf, X_train, y_train, cv=cv, n_jobs=-1).mean()
    std_accuracy = cross_val_score(rf, X_train, y_train, cv=cv, n_jobs=-1).std()

    # Store the results using unpacking
    current_params['mean_accuracy'] = mean_accuracy
    current_params['std_accuracy'] = std_accuracy
    data.append(current_params)

Grid Search: 100%|███████████████████████████████████████████████████████████████████| 784/784 [07:57<00:00,  1.64it/s]


In [77]:
results_1 = pd.DataFrame(data)
results_1 = results_1.sort_values('mean_accuracy', ascending=False)
results_1

,n_estimators,criterion,max_depth,class_weight,random_state,mean_accuracy,std_accuracy
643,100,gini,28,None,21,0.910241,0.016107
439,50,gini,24,None,21,0.910238,0.021260
481,50,gini,45,None,21,0.909500,0.014898
479,50,gini,44,None,21,0.909500,0.014898
477,50,gini,43,None,21,0.909500,0.014898
...,...,...,...,...,...,...,...
1,5,gini,1,None,21,0.366413,0.035357
99,5,entropy,1,None,21,0.346405,0.026789
2,5,gini,2,balanced,21,0.330147,0.037240
98,5,entropy,1,balanced,21,0.328666,0.028446


## 6. Predictions

1. Choose the best model and use it to make predictions for the test dataset.
2. Calculate the final accuracy.

In [81]:
rfc = RandomForestClassifier(n_estimators=100, max_depth = 28, 
                                criterion = "gini", random_state=21)
rfc.fit(X_train, y_train)
predict_rfr = rfc.predict(X_test)
accuracy_score(y_test, predict_rfr)

0.9319526627218935